# Início do projeto

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [ ]:
import seaborn as sn

In [ ]:
import matplotlib.pyplot as plt

In [3]:
BD = pd.read_csv("DMC 2013_task and realclass/transact_train.txt", na_filter=True, na_values='?', sep='|')

In [ ]:
data = {
    'session': BD['sessionNo'],
    'customer': BD['customerNo'],
    'startHour': BD['startHour'],
    'startWeekday': BD['startWeekday'],
    'duration' : BD['duration']
}
p = pd.DataFrame(data,columns=['session','customer', 'startHour','startWeekday','duration'])
p
p.to_csv('saida_1.csv', sep=';', decimal='.', index=False, float_format='%.3f')

In [ ]:
filter_d = p.filter(items=['session','duration'])

In [ ]:
filter_d.stack().index[np.argmax(filter_d.values)]

In [ ]:
#Cria nova função:
def num_missing(x):
  return sum(x.isnull())

#Aplica por linha:
print ("\nMissing values per row:")
print (filter_d.apply(num_missing, axis=1).head() )

In [ ]:
p.loc[(p['session'] == '1') & (p['duration'])]

In [ ]:
## Cria um array de indexes
index = BD['sessionNo'].values
len(index)
index[:20]
# type(index)
# Novo Dataframe
# nData = pd.DataFrame()
# nData

In [ ]:
## Copia apenas as linhas com a ultima sessão por usuario
aux = 0
for idx, item in enumerate(index[:-1]):
    if (index[idx] != index[idx+1]):
#         print("{} - {} ".format(idx, item))
        nData = nData.append(BD.loc[idx:idx], ignore_index=True)
    aux = idx+1
nData = nData.append(BD.loc[aux:aux], ignore_index=True)

In [ ]:
BD.nsmallest(1, 'sessionNo', keep='last')

In [57]:
# Não funciona
# nData = BD.groupby(by='sessionNo').last()
nData = BD.groupby('sessionNo').tail(1)

In [62]:
nData.set_index('sessionNo', inplace=True)

KeyError: "None of ['sessionNo'] are in the columns"

In [68]:
nData.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,completely orderable,1.0,600.0,70.0,21.0,1.0,43.0,1.0,49.0,y
2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,NaN,...,completely orderable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,y
3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,29.99,...,completely orderable,3.0,1800.0,475.0,302.0,12.0,45.0,1.0,11.0,y
4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,4.99,...,completely not orderable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n
5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,27.85,...,completely orderable,4.0,800.0,503.0,18.0,1.0,46.0,1.0,40.0,y


In [71]:
dfTest = nData.copy()

In [45]:
df1 = BD.loc[BD.groupby('sessionNo').duration.idxmax()]

In [46]:
nData2 = BD[BD.groupby('sessionNo').cumcount(ascending=False)==0]

In [47]:
nData2.set_index('sessionNo', inplace=True)

In [49]:
nData2.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            49991, 49992, 49993, 49994, 49995, 49996, 49997, 49998, 49999,
            50000],
           dtype='int64', name='sessionNo', length=50000)

In [52]:
df1.set_index('sessionNo', inplace=True)

In [69]:
nData.equals(nData2)

True

In [56]:
nBool = (df1 != nData2).stack()
nBooldiff = pd.concat([df1.stack()[nBool], nData2.stack()[nBool]], axis=1)

nBooldiff.columns=['old', 'New']
nBooldiff

old     New
sessionNo                          
6         bCount          1       3
          bMaxPrice  129.99     169
          bMinPrice  129.99   99.99
          bSumPrice  129.99  398.98
          cCount          1       3
...                     ...     ...
49953     bMinPrice     169   99.99
          bSumPrice     169  398.98
          cCount          1       3
          cMinPrice     169   99.99
          cSumPrice     169  398.98

[5297 rows x 2 columns]

In [31]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
#     diff_df.to_csv('data/diff1.csv')
    writer = pd.ExcelWriter('data/diff1.xlsx')
    nData.to_excel(writer,'Salvando progresso')
    writer.save()
    return diff_df

In [70]:
dataframe_difference(nData, nData2)

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order,_merge


In [72]:
dfTest is deep

NameError: name 'deep' is not defined

In [ ]:
# nData.to_csv('saida_filtrada_session.csv', sep=';', decimal='.', index=False, float_format='%.4f')

In [7]:
nData.describe()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,bSumPrice,bStep,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,50000.000000,50000.000000,50000.000000,50000.000000,49682.000000,49682.000000,49682.000000,50000.000000,49325.000000,49325.000000,49325.000000,38916.000000,27361.000000,27046.000000,27046.000000,27046.000000,27361.000000,27333.000000,27361.000000,27361.000000
mean,14.367060,5.923140,1465.728502,20.233700,72.118160,157.646205,1096.642791,3.207580,86.444819,118.589753,199.754140,3.044506,11980.285223,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
std,4.638547,0.795063,2314.980566,28.430273,176.181174,295.331281,3083.223860,3.549983,203.739233,232.215900,430.988381,1.633226,7407.139373,2755.134822,131.400171,108.671163,26.743458,12.083048,0.457506,112.447109
min,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.000000,1.000000,3.000000
25%,11.000000,5.000000,217.926750,4.000000,7.000000,24.990000,119.960000,1.000000,9.990000,19.990000,34.950000,1.000000,5478.000000,600.000000,480.000000,36.000000,2.000000,36.000000,1.000000,14.000000
50%,15.000000,6.000000,686.784500,10.000000,14.990000,49.990000,358.935000,2.000000,19.990000,39.990000,74.970000,3.000000,11843.000000,1300.000000,520.000000,98.000000,8.000000,45.000000,2.000000,33.000000
75%,18.000000,7.000000,1726.262750,24.000000,39.990000,129.990000,938.637500,4.000000,49.990000,79.990000,199.970000,5.000000,18431.000000,3200.000000,553.000000,215.000000,14.000000,53.000000,2.000000,85.000000
max,23.000000,7.000000,21580.092000,200.000000,5999.990000,6999.990000,115742.000000,108.000000,6999.990000,6999.990000,23116.880000,5.000000,25038.000000,50000.000000,638.000000,600.000000,868.000000,99.000000,3.000000,738.000000


In [64]:
BD.groupby('sessionNo').get_group(2).mean()

sessionNo           2.000000
startHour           6.000000
startWeekday        5.000000
duration           41.507400
cCount              0.000000
cMinPrice                NaN
cMaxPrice                NaN
cSumPrice                NaN
bCount              0.000000
bMinPrice                NaN
bMaxPrice                NaN
bSumPrice                NaN
bStep               3.333333
customerNo               NaN
maxVal                   NaN
customerScore            NaN
accountLifetime          NaN
payments                 NaN
age                      NaN
address                  NaN
lastOrder                NaN
dtype: float64

In [8]:
## Novas variaveis
xpto = BD.groupby('sessionNo')['cCount'].mean()
xpto2 = BD.groupby('sessionNo')['bCount'].mean()
xpto3 = BD.groupby('sessionNo')['bSumPrice'].mean()
xpto4 = BD.groupby('sessionNo')['bStep'].median()
xpto5 = BD.groupby('sessionNo')['duration'].mean()

In [9]:
xpto.describe()

count    50000.000000
mean        15.544077
std         20.370774
min          0.000000
25%          3.000000
50%          8.000000
75%         19.000000
max        198.000000
Name: cCount, dtype: float64

In [10]:
## Criando novas colunas com a Média
dfTest['media_cCount'] = xpto
dfTest['media_bCount'] = xpto2
dfTest['media_bSumPrice'] = xpto3
dfTest['media_bStep'] = xpto4
dfTest['media_duration'] = xpto5

In [124]:
x = dfTest.filter(regex="[^order]")
x.columns
y = dfTest.iloc[:,22]
y.columns

sessionNo
1        y
3        y
5        y
7        y
12       y
        ..
49987    y
49990    y
49991    y
49993    n
49995    n
Name: order, Length: 22353, dtype: object

In [149]:
# b = dfTest.filter(regex="[(^order | ^sessionNo]")
# b['order']
dfTest.columns.difference(['order'])

27

In [80]:
BD.groupby('sessionNo')['cCount'].mean()

sessionNo
1          1.000000
2          0.000000
3         10.600000
4          4.000000
5         26.875000
            ...    
49996    172.935484
49997      8.000000
49998     44.333333
49999      6.500000
50000      1.000000
Name: cCount, Length: 50000, dtype: float64

In [125]:
previsores = x.values
classe = y.values

In [126]:
# Removendo linhas nulas
dfTest.isna().sum()
# dfTest.dropna(axis=0, subset=['bSumPrice', 'age'], inplace=True)
# dfTest.dropna(inplace=True)

startHour          0
startWeekday       0
duration           0
cCount             0
cMinPrice          0
cMaxPrice          0
cSumPrice          0
bCount             0
bMinPrice          0
bMaxPrice          0
bSumPrice          0
bStep              0
onlineStatus       0
availability       0
customerNo         0
maxVal             0
customerScore      0
accountLifetime    0
payments           0
age                0
address            0
lastOrder          0
order              0
media_cCount       0
media_bCount       0
media_bSumPrice    0
media_bStep        0
media_duration     0
dtype: int64

In [12]:
## Tratando os valores ausentes (Moda)- onlineStatus 
dfTest.loc[pd.isnull(dfTest['onlineStatus']), 'onlineStatus'] = dfTest['onlineStatus'].mode().values[0]
dfTest.loc[ dfTest['onlineStatus'] == 'y' ]

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,payments,age,address,lastOrder,order,media_cCount,media_bCount,media_bSumPrice,media_bStep,media_duration
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,1.0,43.0,1.0,49.0,y,1.000000,1.000000,59.990000,2.0,17.275667
2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,y,0.000000,0.000000,NaN,4.0,41.507400
3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,29.99,...,12.0,45.0,1.0,11.0,y,10.600000,1.800000,37.982000,4.0,291.070600
4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,4.99,...,NaN,NaN,NaN,NaN,n,4.000000,1.000000,4.990000,1.0,35.841500
5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,27.85,...,1.0,46.0,1.0,40.0,y,26.875000,3.250000,85.600000,4.0,1538.791250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,18,7,7271.812,197,6.99,59.99,4315.03,13,9.99,54.99,...,NaN,NaN,NaN,NaN,y,172.935484,10.483871,173.885161,1.0,6301.146774
49997,18,7,667.219,8,70.99,89.99,667.88,1,89.99,89.99,...,NaN,NaN,NaN,NaN,n,8.000000,1.000000,89.990000,2.0,654.031500
49998,18,7,5988.882,77,9.99,149.99,5253.28,3,49.95,99.99,...,NaN,NaN,NaN,NaN,n,44.333333,2.000000,143.300000,NaN,4550.391333


In [13]:
## Substituir os valores y e n por 1 e 0, respectivamente
dfTest['onlineStatus'] = dfTest.apply(lambda x: 1 if (x['onlineStatus']=='y') else 0, axis=1)
dfTest['onlineStatus']

sessionNo
1        1
2        1
3        1
4        1
5        1
        ..
49996    1
49997    1
49998    1
49999    1
50000    1
Name: onlineStatus, Length: 50000, dtype: int64

In [15]:
## Tratando os valores ausentes (Moda)- availability
dfTest.loc[pd.isnull(dfTest['availability']), 'availability'] = dfTest['availability'].mode().values[0]
dfTest['availability']
dfTest.loc[pd.isnull(dfTest['availability']), 'availability']

Series([], Name: availability, dtype: object)

In [16]:
## Tratando os valores ausentes em customerNo - Apagar os registros ausentes
dfTest.drop(dfTest[pd.isnull(dfTest['customerNo'])].index, inplace=True)

In [69]:
overY = dfTest.loc[ (dfTest['bSumPrice'] > dfTest['maxVal']) & (dfTest['order'] == 'y'), 'order' ]

In [41]:
dfTest

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,payments,age,address,lastOrder,order,media_cCount,media_bCount,media_bSumPrice,media_bStep,media_duration
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,1.0,43.0,1.0,49.0,y,1.000000,1.000000,59.990000,2.0,17.275667
3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,29.99,...,12.0,45.0,1.0,11.0,y,10.600000,1.800000,37.982000,4.0,291.070600
5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,27.85,...,1.0,46.0,1.0,40.0,y,26.875000,3.250000,85.600000,4.0,1538.791250
7,6,5,973.887,15,3.00,40.00,137.00,3,3.00,3.00,...,10.0,31.0,2.0,10.0,y,9.933333,1.933333,5.800000,1.5,547.752333
12,6,5,1540.895,21,5.99,59.99,456.77,11,5.99,59.99,...,11.0,86.0,2.0,37.0,y,15.185185,7.740741,175.554815,3.0,821.752593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49987,18,7,336.390,13,19.99,199.99,239.97,1,19.99,19.99,...,29.0,43.0,2.0,7.0,y,13.000000,1.000000,19.990000,3.5,293.469200
49990,18,7,228.902,8,9.99,19.99,115.92,8,9.99,19.99,...,13.0,41.0,2.0,41.0,y,5.846154,5.846154,83.864615,3.0,52.640538
49991,18,7,3895.061,47,3.00,29.99,449.80,4,5.00,17.99,...,13.0,58.0,2.0,164.0,y,37.944444,3.611111,42.749444,1.5,2975.704222


In [ ]:
import matplotlib.pyplot as plt
import seaborn as snsplt.figure(figsize=(10, 7))
sns.heatmap(df.corr(),
            annot = True,
            fmt = '.2f',
            cmap='Blues')
plt.title('Correlação entre variáveis do dataset de Iris')
plt.show()

In [121]:
# Eliminar as caracteristicas não-numericas ou substitui-las
# for idx, item in enumerate(dfTest['availability'].unique()):
#     print("{} - {}".format(idx, item))
    
# l = np.ndenumerate(dfTest['availability'].unique())
# l
# avalibility = [ idx[0][0] for idx in l ]
# avalibility = list(map(lambda x: x[0][0], l))
dfTest['availability']
# avalibility

# li = dfTest['availability'].unique().tolist()
# li.index('mixed')
# test = list(map( lambda x: li.index(x), dfTest['availability'].values))
# test.count(0)
# len(test)
# dfTest['availability'] = test
# from collections import Counter
# Counter(test)

sessionNo
1        0
3        0
5        0
7        0
12       0
        ..
49987    0
49990    0
49991    0
49993    0
49995    0
Name: availability, Length: 22353, dtype: int64

In [42]:
dfTest.query( 'cCount == 0 & bCount == 0' )

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,payments,age,address,lastOrder,order,media_cCount,media_bCount,media_bSumPrice,media_bStep,media_duration
sessionNo,,,,,,,,,,,,,,,,,,,,,


In [213]:
writer = pd.ExcelWriter('nDataExcel.xlsx')
nData.to_excel(writer,'Salvando progresso')
writer.save()

# DF TO CSV
nData.to_csv('nDataExport.csv', sep='|', decimal='.')

In [215]:
# binario
nData.to_pickle('nData_bin')

In [122]:
from sklearn.impute import SimpleImputer

In [127]:
imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
imputer = imputer.fit(previsores)
previsores = imputer.transform(previsores)

In [166]:
from sklearn.preprocessing import StandardScaler

In [167]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [168]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(previsores, classe, test_size=0.3)

In [171]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(random_state=42, n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [172]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8571428571428571


In [173]:
# A implementação do Cross Validation
from sklearn.model_selection import cross_val_score

#Create a Gaussian Classifier
clf_d=RandomForestClassifier(random_state=0, n_estimators=100)

scores_dt = cross_val_score(clf_d, X_train, y_train,
                            scoring='accuracy', cv=5)

print(scores_dt.mean())

0.8640629647809904
